This notebook is focused on the modelling aspect of the project.  It predominantly depends on the modules of [src/modelling](./src/modelling).

<br>

## Preliminaries

### Paths

In [1]:
import os
import pathlib
import sys

In [2]:
if not 'google.colab' in str(get_ipython()):
    
    parts = pathlib.Path(os.getcwd()).parts    
    limit = max([index for index, value in enumerate(parts) if value == 'infections'])    
    parent = os.path.join(*list(parts[:(limit + 1)]))
    
    sys.path.append(os.path.join(parent, 'src'))


In [3]:
parent

'J:\\library\\premodelling\\projects\\infections'

<br>
<br>

### Libraries

In [4]:
%matplotlib inline

import datetime

import logging
import collections

import numpy as np
import pandas as pd


<br>

### Custom

In [5]:
import src.modelling.DataStreams
import src.modelling.DataReconstructions
import src.modelling.Differences
import src.modelling.DataNormalisation
import src.modelling.Estimates

<br>
<br>

### Logging

In [6]:
logging.basicConfig(level=logging.INFO,
                    format='\n\n%(message)s\n%(asctime)s.%(msecs)03d',
                        datefmt='%Y-%m-%d %H:%M:%S')
logger = logging.getLogger(__name__)

<br>
<br>

## Part II

### Setting-Up

A class for the data splitting fractions

In [7]:
Fraction = collections.namedtuple(
    typename='Fraction',
    field_names=['training', 'validating', 'testing'])

<br>

**Modelling Arguments**

> * Predict `output_steps` days into the future, based on `input_width` days of history

Herein

* $input\_width \in widths$  $\qquad$  [$widths$ is a range of input window values (days)]
* $output\_steps = 15$ days
  
And

* $label\_width = output\_steps$


In [8]:
Arguments = collections.namedtuple(
    typename='Arguments',
    field_names=['input_width', 'label_width', 'shift', 'training_', 'validating_', 'testing_', 'label_columns'])

<br>

Vary the widths' range as required.

In [9]:
widths = range(18, 23)
output_steps = 15

<br>
<br>

### Training, Validating, Testing Data

Foremost: The data sets for training, validating, and testing

In [10]:
training, validating, testing = src.modelling.DataStreams.DataStreams(root=parent, fraction=Fraction._make(
        (0.75, 0.15, 0.10))).exc()

In [11]:
logger.info(training.columns)



Index(['group', 'date', 'covidOccupiedBeds', 'covidOccupiedMVBeds',
       'estimatedNewAdmissions', 'EDC0-4', 'EDC5-9', 'EDC10-14', 'EDC15-19',
       'EDC20-24', 'EDC25-29', 'EDC30-34', 'EDC35-39', 'EDC40-44', 'EDC45-49',
       'EDC50-54', 'EDC55-59', 'EDC60-64', 'EDC65-69', 'EDC70-74', 'EDC75-79',
       'EDC80-84', 'EDC85-89', 'EDC90+', 'newDeaths28DaysByDeathDate',
       'EDV12-15', 'EDV16-17', 'EDV18-24', 'EDV25-29', 'EDV30-34', 'EDV35-39',
       'EDV40-44', 'EDV45-49', 'EDV50-54', 'EDV55-59', 'EDV60-64', 'EDV65-69',
       'EDV70-74', 'EDV75-79', 'EDV80-84', 'EDV85-89', 'EDV90+'],
      dtype='object')
2022-03-16 20:32:54.869


<br>
<br>

### Reconstruction

Reconstructions: Each data set is a concatenation of records from various NHS Trusts, however because the aim is a single predicting/forecasting model for all trusts, the data should be reconstructed ...

In [12]:
reconstructions = src.modelling.DataReconstructions.DataReconstructions()
training = reconstructions.exc(blob=training)
validating = reconstructions.exc(blob=validating)
testing = reconstructions.exc(blob=testing)

<br>
<br>

### Differences

Using difference values rather than actual values

In [13]:
differences = src.modelling.Differences.Differences()
training = differences.exc(blob=training)
validating = differences.exc(blob=validating)
testing = differences.exc(blob=testing)

<br>
<br>

### Normalisation

In [14]:
normalisation = src.modelling.DataNormalisation.DataNormalisation(reference=training)
training_ = normalisation.normalise(blob=training)
validating_ = normalisation.normalise(blob=validating)
testing_ = normalisation.normalise(blob=testing)

training_.drop(columns='point', inplace=True)
validating_.drop(columns='point', inplace=True)
testing_.drop(columns='point', inplace=True)

<br>
<br>

### Modelling

In [15]:
arguments = Arguments(input_width=None, label_width=output_steps, shift=output_steps,
                      training_=training_, validating_=validating_, testing_=testing_,
                      label_columns=['estimatedNewAdmissions'])

validations, tests = src.modelling.Estimates.Estimates(
    n_features=training_.shape[1], 
    output_steps=output_steps).exc(widths=widths, arguments=arguments)

logger.info(validations)
logger.info(tests)

Epoch 1/100
2423/2423 [==============================] - 13s 5ms/step - loss: 6.6073 - mean_absolute_error: 0.9745 - val_loss: 0.5908 - val_mean_absolute_error: 0.5511
Epoch 2/100
2423/2423 [==============================] - 13s 5ms/step - loss: 1.1031 - mean_absolute_error: 0.5690 - val_loss: 0.5469 - val_mean_absolute_error: 0.5267
Epoch 3/100
2423/2423 [==============================] - 12s 5ms/step - loss: 1.1521 - mean_absolute_error: 0.5690 - val_loss: 0.5549 - val_mean_absolute_error: 0.5298
Epoch 4/100
2423/2423 [==============================] - 12s 5ms/step - loss: 1.1787 - mean_absolute_error: 0.5762 - val_loss: 0.5404 - val_mean_absolute_error: 0.5221
Epoch 5/100
2423/2423 [==============================] - 13s 5ms/step - loss: 1.1025 - mean_absolute_error: 0.5528 - val_loss: 0.5338 - val_mean_absolute_error: 0.5184
Epoch 6/100
2423/2423 [==============================] - 12s 5ms/step - loss: 1.0672 - mean_absolute_error: 0.5387 - val_loss: 0.5313 - val_mean_absolute_error:



  method history ahead      loss       mae
0    CNN      18    15  0.550707  0.528657
1   LSTM      18    15  0.538723  0.527325
2    CNN      19    15  0.540729  0.521727
3   LSTM      19    15  0.530134  0.519053
4    CNN      20    15  0.618428  0.554454
5   LSTM      20    15  0.529472  0.519232
6    CNN      21    15  0.545342  0.525292
7   LSTM      21    15  0.534071  0.523419
8    CNN      22    15  0.560468  0.532892
9   LSTM      22    15  0.539802  0.527458
2022-03-16 21:17:42.418


  method history ahead      loss       mae
0    CNN      18    15  0.894029  0.391921
1   LSTM      18    15  0.717038  0.347893
2    CNN      19    15  0.749748  0.349158
3   LSTM      19    15  0.694559  0.342238
4    CNN      20    15   1.35914  0.504816
5   LSTM      20    15  0.694891  0.346347
6    CNN      21    15  0.753045  0.349489
7   LSTM      21    15  0.696939   0.33825
8    CNN      22    15   0.86576  0.388487
9   LSTM      22    15  0.696852  0.350093
2022-03-16 21:17:42.426


<br>
<br>

### Delete DAG Diagrams

In [16]:
%%bash

rm -rf *.pdf